In [1]:
# !pip install pyjwt==1.6.4
import os
import jwt
import uuid
import hashlib
from urllib.parse import urlencode
import pandas as pd
import matplotlib.pyplot as plt

import requests

In [ ]:
def getCoinDF(market, timedelta, count):
    url = f"https://api.upbit.com/v1/candles/minutes/{timedelta}"

    query = {"market":f"{market}", "count":f"{count}"}

    response = requests.request("GET", url, params=query)

    df = pd.read_json(response.text)

    while(len(df)<100):
        if len(df)%1000==0:
            print(len(df))
        ts = df['timestamp'].iloc[-1]
        last = f'{ts.year:02d}-{ts.month:02d}-{ts.day:02d} {ts.hour:02d}:{ts.minute:02d}:{round(ts.second):02d}'
        url = f"https://api.upbit.com/v1/candles/minutes/{timedelta}"

        querystring = {"market":f"{market}","count":f"{count}",'to':last}

        response = requests.request("GET", url, params=querystring)

        if response.text == "Too many API requests." :
            continue

        df_app = pd.read_json(response.text)
        df = pd.concat([df,df_app], ignore_index=True)

    return df


#coins = ['KRW-BTC','KRW-ETH','KRW-XRP']
coins = ['KRW-ETH']
df = {}
for coin in coins:
    print(coin)
    df[coin] = getCoinDF(coin,30,200)
  
df['KRW-ETH']

In [ ]:
df['KRW-ETH'] = df['KRW-ETH'].loc[::-1,:]
df['KRW-ETH'] = df['KRW-ETH'].reset_index()

In [ ]:
train_index = -30
series = df['KRW-ETH']['opening_price'][-30+train_index:train_index].reset_index()['opening_price']
test = df['KRW-ETH']['opening_price'][train_index:train_index+1].reset_index()['opening_price']

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()


model.add(layers.LSTM(128))

model.add(layers.Dense(256, activation='linear'))
  
model.add(tf.keras.layers.Dense(1))

model.compile('Adam', loss='mse')